In [361]:
import json
import os

import numpy as np
import pandas as pd
from tqdm import tqdm
import re
import warnings
warnings.filterwarnings("ignore")

In [366]:
df_og = pd.read_csv('review_data.csv')

In [367]:
df = df_og[0:1000]

In [368]:
df

,Unnamed: 0,stars,text
0,0,1.0,Total bill for this horrible service? Over $8G...
1,1,5.0,I *adore* Travis at the Hard Rock's new Kelly ...
2,2,5.0,I have to say that this office really has it t...
3,3,5.0,Went in for a lunch. Steak sandwich was delici...
4,4,1.0,Today was my second out of three sessions I ha...
...,...,...,...
995,995,4.0,We ate there yesterday for the first time. My ...
996,996,2.0,I have never waited LESS THAN forty five minut...
997,997,5.0,I am short. Pants are always waaaay too long. ...
998,998,4.0,"Great ambiance, great food, although their pol..."


In [369]:
df.to_csv('review_data.csv')

In [249]:
df_og['text'][0]

'Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.'

In [250]:
for i in range(50):
    print(df['text'][i])

Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.
I *adore* Travis at the Hard Rock's new Kelly Cardenas Salon!  I'm always a fan of a great blowout and no stranger to the chains that offer this service; however, Travis has taken the flawless blowout to a whole new level!  

Travis's greets you with his perfectly green swoosh in his otherwise perfectly styled black hair and a Vegas-worthy rockstar outfit.  Next comes the most relaxing and incredible shampoo -- where you get a full head message that could cure even the very worst migraine in minutes --- and the scented shampoo room.  Travis has freakishly strong fingers (in a good way) and use the perfect amount of pressure.  That was superb!  Then starts the glorious blowout... where not one, not two, but THREE people were involved in doing the best round-brush action my hair has eve

In [251]:
def clean_string(string):
    """
    Removes unsightly html code in a given string
    """
    string = string.lower()
    string = re.sub(r"http\S+", "", string)
#     clean0 = re.compile(r'</code></pre>')
#     out = re.sub(clean0,' ', string)
    clean = re.compile('<.*?>')
    out = re.sub(clean,' ', string)
    clean1 = r'^https?:\/\/.*[\r\n]*'
    out = re.sub(clean1,' ', out)
    clean2 = re.compile(r'[^\w\s]')
    out = re.sub(clean2,' ', out)
    return out
CONTRACTION_MAP = {
"ain't": "is not","aren't": "are not","can't": "cannot","can't've": "cannot have","'cause": "because",
"could've": "could have","couldn't": "could not","couldn't've": "could not have","didn't": "did not",
"doesn't": "does not","don't": "do not","hadn't": "had not","hadn't've": "had not have","hasn't": "has not",
"haven't": "have not","he'd": "he would","he'd've": "he would have","he'll": "he will","he'll've": "he he will have",
"he's": "he is","how'd": "how did","how'd'y": "how do you","how'll": "how will","how's": "how is","I'd": "I would",
"I'd've": "I would have","I'll": "I will","I'll've": "I will have","I'm": "I am","I've": "I have","i'd": "i would",
"i'd've": "i would have","i'll": "i will","i'll've": "i will have","i'm": "i am","i've": "i have","isn't": "is not",
"it'd": "it would","it'd've": "it would have","it'll": "it will","it'll've": "it will have","it's": "it is",
"let's": "let us","ma'am": "madam","mayn't": "may not","might've": "might have","mightn't": "might not",
"mightn't've": "might not have","must've": "must have","mustn't": "must not","mustn't've": "must not have",
"needn't": "need not","needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
"oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not","shan't've": "shall not have",
"she'd": "she would","she'd've": "she would have","she'll": "she will","she'll've": "she will have","she's": "she is",
"should've": "should have","shouldn't": "should not","shouldn't've": "should not have","so've": "so have",
"so's": "so as","that'd": "that would","that'd've": "that would have","that's": "that is","there'd": "there would",
"there'd've": "there would have","there's": "there is","they'd": "they would","they'd've": "they would have",
"they'll": "they will","they'll've": "they will have","they're": "they are","they've": "they have","to've": "to have",
"wasn't": "was not","we'd": "we would","we'd've": "we would have","we'll": "we will","we'll've": "we will have",
"we're": "we are","we've": "we have","weren't": "were not","what'll": "what will","what'll've": "what will have",
"what're": "what are","what's": "what is","what've": "what have","when's": "when is","when've": "when have",
"where'd": "where did","where's": "where is","where've": "where have","who'll": "who will",
"who'll've": "who will have","who's": "who is","who've": "who have","why's": "why is","why've": "why have",
"will've": "will have","won't": "will not","won't've": "will not have","would've": "would have",
"wouldn't": "would not","wouldn't've": "would not have","y'all": "you all","y'all'd": "you all would",
"y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would",
"you'd've": "you would have","you'll": "you will","you'll've": "you will have","you're": "you are","you've": "you have"
}
def expand_contractions(text, contraction_mapping=CONTRACTION_MAP):
    """
    Expands any contractions within a given string
    """

    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())), 
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

In [252]:
df['text']=df['text'].apply(clean_string)
df['text']=df['text'].apply(expand_contractions)

In [253]:
#import natural language toolkit
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
analyser = SentimentIntensityAnalyzer()
from nltk.probability import FreqDist
import string

## import word tokenizer
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer 

# get lexicon 
nltk.download('vader_lexicon') 
nltk.download('punkt')
nltk.download('stopwords')

#specify dataframe here
nltk_data = df

# tokenize, stem, and remove stop words
nltk_data['token'] = nltk_data['text'].apply(word_tokenize)
stop_words = set(stopwords.words('english'))
nltk_data['stop'] = nltk_data['token'].apply(lambda x: [item for item in x if item not in stop_words])
nltk_data['stem'] = nltk_data['stop'].apply(lambda x: [ps.stem(y) for y in x])
stem_nltk_data = nltk_data['stem'].apply(pd.Series).stack()

# calculate word frequency
fdist_nltk_data = FreqDist(stem_nltk_data)

# remove punctuation counts
for punc in string.punctuation:
    del fdist_nltk_data[punc]

# get list of the 'content' (responses) 
nltk_data_list = list(nltk_data['text'].values)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\chang\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\chang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [254]:
fdist_nltk_data

FreqDist({'place': 576, 'good': 565, 'food': 562, 'time': 490, 'great': 467, 'go': 437, 'like': 429, 'get': 421, 'servic': 396, 'one': 374, ...})

In [255]:
nltk_data_sentiments = get_sentiments(nltk_data_list)
nltk_data_sent = nltk_data.drop(['token','stop'], axis=1).assign(compound=nltk_data_sentiments['compound'])

NameError: name 'get_sentiments' is not defined

In [ ]:
nltk_data_sent

In [ ]:
# import matplotlib.pyplot as plt
# plt.scatter(nltk_data_sent['stars'],nltk_data_sent['compound'])

In [ ]:
def get_sentiments(input_list):
    """
    output dataframe that stores sentiment information
    """
    output = pd.DataFrame()
    
    for sentence in input_list:
        ss = analyser.polarity_scores(sentence)
        ss['sentence'] = sentence
        output = output.append(ss, ignore_index=True)

    return output

In [ ]:
nltk_data_list 

In [ ]:
def check_sentiment():
    
    # list to store user post if they intend to post
    msg_list = []
    
    #create sentiment dataframe here
    msg = input('Message :\t')
    d = {'content': [msg]}
    df = pd.DataFrame(data=d)
    
    # specify data for sentiment analysis
    nltk_data = df
    
    # tokenize, stem, and remove stop words
    nltk_data['token'] = nltk_data['content'].apply(word_tokenize)
    stop_words = set(stopwords.words('english'))
    nltk_data['stop'] = nltk_data['token'].apply(lambda x: [item for item in x if item not in stop_words])
    nltk_data['stem'] = nltk_data['stop'].apply(lambda x: [ps.stem(y) for y in x])
    stem_nltk_data = nltk_data['stem'].apply(pd.Series).stack()
    
    # get list of the 'content' (responses) 
    nltk_data_list = list(nltk_data['content'].values)
    
    # get sentiments
    nltk_data_sentiments = get_sentiments(nltk_data_list)
    
    # alert user of sentiment score
    print('Sentiment Score:', nltk_data_sentiments['compound'])


In [71]:
check_sentiment()

Message :	what doesnt like this bitch
Sentiment Score: 0   -0.7657
Name: compound, dtype: float64


In [17]:
fdist_nltk_data.value

NameError: name 'fdist_nltk_data' is not defined

In [256]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [352]:
%%time
def get_tag(lst,out_put={}):
    if lst==[]:
        None
    else: 
        if lst[0] not in out_put.keys():

            out_put.update({ lst[0] : str(nlp(lst[0])[0].pos_)})

            get_tag(lst[1:],out_put)
        else: 
            get_tag(lst[1:],out_put)

Wall time: 0 ns


In [353]:
text="I ditched my motorcycle after I crashed on the 805 south. Then I got hit by a DUI driver"
text=text.split(' ')

In [355]:
%%time
out_put ={}
get_tag(text,out_put=out_put)
out_put

Wall time: 60.3 ms


{'I': 'PRON',
 'ditched': 'VERB',
 'my': 'PRON',
 'motorcycle': 'NOUN',
 'after': 'ADP',
 'crashed': 'VERB',
 'on': 'ADP',
 'the': 'DET',
 '805': 'NUM',
 'south.': 'ADV',
 'Then': 'ADV',
 'got': 'VERB',
 'hit': 'VERB',
 'by': 'ADP',
 'a': 'DET',
 'DUI': 'PROPN',
 'driver': 'NOUN'}

Wall time: 63.8 ms


{'i': 'PRON',
 'ditched': 'VERB',
 'my': 'PRON',
 'motorcycle': 'NOUN',
 'after': 'ADP',
 'crashed': 'VERB',
 'on': 'ADP',
 'the': 'DET',
 '805': 'NUM',
 'south ': 'ADV',
 'then': 'ADV',
 'got': 'VERB',
 'hit': 'VERB',
 'by': 'ADP',
 'a': 'DET',
 'dui': 'NOUN',
 'driver': 'NOUN'}

In [230]:
text=['hi']

In [174]:
lst[1:]

[]

In [275]:
text=[]
for i in df_og[0:10]['text']:
    i=clean_string(i)
    i=i.split(' ')
    text.extend(i)
text_=[]
for i in text:
    if i=='':
        continue
    else:
        text_.append(i)

In [276]:
text_

['total',
 'bill',
 'for',
 'this',
 'horrible',
 'service',
 'over',
 '8gs',
 'these',
 'crooks',
 'actually',
 'had',
 'the',
 'nerve',
 'to',
 'charge',
 'us',
 '69',
 'for',
 '3',
 'pills',
 'i',
 'checked',
 'online',
 'the',
 'pills',
 'can',
 'be',
 'had',
 'for',
 '19',
 'cents',
 'each',
 'avoid',
 'hospital',
 'ers',
 'at',
 'all',
 'costs',
 'i',
 'adore',
 'travis',
 'at',
 'the',
 'hard',
 'rock',
 's',
 'new',
 'kelly',
 'cardenas',
 'salon',
 'i',
 'm',
 'always',
 'a',
 'fan',
 'of',
 'a',
 'great',
 'blowout',
 'and',
 'no',
 'stranger',
 'to',
 'the',
 'chains',
 'that',
 'offer',
 'this',
 'service',
 'however',
 'travis',
 'has',
 'taken',
 'the',
 'flawless',
 'blowout',
 'to',
 'a',
 'whole',
 'new',
 'level',
 '\n\ntravis',
 's',
 'greets',
 'you',
 'with',
 'his',
 'perfectly',
 'green',
 'swoosh',
 'in',
 'his',
 'otherwise',
 'perfectly',
 'styled',
 'black',
 'hair',
 'and',
 'a',
 'vegas',
 'worthy',
 'rockstar',
 'outfit',
 'next',
 'comes',
 'the',
 'most'